数据集共有60000张，分割成50000张训练集10000张测试集
`lambda`是匿名函数的标志，比如一个函数正常的时候这样写：
```
def f(x):
    a = x**2
    return a
a = f(4)
```
也可以简单的写成：
```g = lambda x: x**2 a = g(4)```
`map(function, iterable, ...)`是将`iterable`的数值依次放入`function`里面，比如下面例子：

```map(lambda x: x ** 2, [1, 2, 3, 4, 5])  
结果：
[1, 4, 9, 16, 25]
```


In [5]:
import os, random, time
def get_imgs(rate=0.2):
    """
    获取图片，并划分训练集和测试集
    参数：
        rate:测试集和训练集的比例，rate=测试集个数/训练集个数
    返回值：
        test_imgs:测试集
        test_labels:测试集标签
        train_imgs:训练集
        train_labels:训练集标签
    """
    data_path = './Discuz'
    #读取图片
    imgs = os.listdir(data_path)
    #打乱图片的顺序
    random.shuffle(imgs)
    #计算出数据量
    imgs_num = len(imgs)
    #按照rate比例算出测试集个数
    test_num = int(imgs_num * rate / (1 + rate))
    #测试集数据,从打乱的数据集里面取前test_num个数据
    test_imgs = imgs[:test_num]
    #根据文件名字来获得测试集标签
    #文件名为`a2ai.jpg`
    test_labels = list(map(lambda x: x.split('.')[0], test_imgs))
    #训练集
    train_imgs = imgs[test_num:]
    #训练集标签
    train_labels = list(map(lambda x: x.split('.')[0], train_imgs))
    return test_imgs, test_labels, train_imgs, train_labels

标签向量化，以及向量化的标签进行还原

|字符|ASCII码|
|-|-|
|0|48|
|9|57|
|A|65|
|Z|90|
|a|97|
|z|122|

In [12]:
import numpy as np
def text2vec(text):
    """
    文本转向量
    参数：
        text:文本 例如`a1df`
    返回值：
        vector:向量
    """
    if len(text) > 4:
        raise ValueError('验证码最长4个字符')
    
    vector = np.zeros(4 * 63)
    def char2pos(c):
        if c == '_':
            k = 62
            return k
        k = ord(c) - 48 #判断是否为数字
        if k > 9:
            k = ord(c) - 55 #判断是否为大写字母
            if k > 35:
                k = ord(c) - 61 #判断是否为小写字母
                if k > 61:
                    raise ValueError('No map')
        return k
    for i, c in enumerate(text):
        idx = i * 63 + char2pos(c)
        vector[idx] = 1
    return vector

def vec2text(vec):
    """
    向量转为文本
    参数：
        vec:向量
    返回值：
        文本
    """
    char_pos = vec.nonzero()[0]
    text = []
    for i, c in enumerate(char_pos):
        char_at_pos = i #c/63
        char_idx = c % 63
        if char_idx < 10:
            char_code = char_idx + ord('0')
        elif char_idx < 36:
            char_code = char_idx - 10 + ord('A')
        elif char_idx < 62:
            char_code = char_idx - 36 + ord('a')
        elif char_idx == 62:
            char_code = ord('_')
        else:
            raise ValueError('error')
        text.append(chr(char_code))
    return "".join(text)
print(text2vec('abcd'))
print(vec2text(text2vec('abcd')))

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
abcd


根据batch_size获取数据，从数据集中去除batch_size的数据

TabError: inconsistent use of tabs and spaces in indentation (<ipython-input-15-b03271aa1611>, line 15)